In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
from cv2 import cv2
import os
sns.set()

In [2]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    AspectAwarePreprocessor, MeanPreprocessor, MeanSubtractionPreProcessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet, MiniGoogleNet
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:
from loader_util.callbacks import TrainingMonitor
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from typing import List

In [4]:
# define the total no of epochs
NUM_EPOCHS = 70
INIT_LR = 5e-3
batch_size = 64

# define the polynomial learnign rate scheduler
def poly_decay(epoch):
    # initialise the max no of epochs
    max_epochs = NUM_EPOCHS
    base_lr = INIT_LR
    power = 1.0
    
    # compute new learnign enumerate
    alpha = base_lr * (1 - (epoch / float(max_epochs)))**power
    
    # return the new lr
    return alpha

In [5]:
# load the train test set
(trainx, trainy), (testx, testy) = cifar10.load_data()
trainx = trainx.astype('float')
testx = testx.astype('float')

In [6]:
mp = MeanSubtractionPreProcessor()

def preprocess_image(dataset: List[np.ndarray]):
    return_data = []
    
    for image in dataset:
        processed_image = mp.preprocess(image)
        return_data.append(processed_image)
        
    return np.array(return_data)

In [7]:
# apply normalisation to images
trainx = preprocess_image(trainx)
testx = preprocess_image(testx)

In [8]:
# convert the labels from integers to vectors
lb = LabelBinarizer()
trainy = lb.fit_transform(trainy)
testy = lb.transform(testy)

In [9]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [10]:
# construct the set of callbacks
fig_path = os.path.sep.join(['./', f"{os.getpid()}.png"])
json_path = os.path.sep.join(['./', f"{os.getpid()}.json"])
callbacks = [TrainingMonitor(fig_path, json_path),
             LearningRateScheduler(poly_decay)]

In [11]:
# initialise the optimizer model
print(f'[INFO] Compiling Model....')
opt = SGD(lr=INIT_LR, momentum=0.9)
model = MiniGoogleNet.build(width=32,
                            height=32,
                            depth=3,
                            classes=10)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

[INFO] Compiling Model....
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
# train the network
print('[INFO] training network....')
model.fit_generator(aug.flow(trainx, trainy, batch_size=batch_size),
                    validation_data=(testx, testy),
                    steps_per_epoch=len(trainx) // batch_size,
                    epochs=NUM_EPOCHS,
                    callbacks=callbacks, verbose=1)

[INFO] training network....
Instructions for updating:
Use tf.cast instead.
Epoch 1/70
782/782 [==============================] - 89s 114ms/step - loss: 1.4955 - acc: 0.4630 - val_loss: 1.5403 - val_acc: 0.4918
Epoch 2/70
782/782 [==============================] - 85s 109ms/step - loss: 1.0759 - acc: 0.6215 - val_loss: 1.2283 - val_acc: 0.5717
Epoch 3/70
782/782 [==============================] - 86s 109ms/step - loss: 0.8990 - acc: 0.6845 - val_loss: 0.8966 - val_acc: 0.6846
Epoch 4/70
782/782 [==============================] - 86s 110ms/step - loss: 0.7944 - acc: 0.7230 - val_loss: 1.0079 - val_acc: 0.6689
Epoch 5/70
782/782 [==============================] - 86s 110ms/step - loss: 0.7108 - acc: 0.7535 - val_loss: 0.8851 - val_acc: 0.6949
Epoch 6/70
782/782 [==============================] - 86s 110ms/step - loss: 0.6545 - acc: 0.7725 - val_loss: 0.8312 - val_acc: 0.7301
Epoch 7/70
782/782 [==============================] - 86s 109ms/step - loss: 0.6077 - acc: 0.7916 - val_loss: 0.78

Epoch 36/70
782/782 [==============================] - 86s 109ms/step - loss: 0.1735 - acc: 0.9405 - val_loss: 0.5038 - val_acc: 0.8557
Epoch 37/70
782/782 [==============================] - 86s 109ms/step - loss: 0.1610 - acc: 0.9438 - val_loss: 0.4858 - val_acc: 0.8657
Epoch 38/70
782/782 [==============================] - 86s 110ms/step - loss: 0.1597 - acc: 0.9448 - val_loss: 0.5256 - val_acc: 0.8424
Epoch 39/70
782/782 [==============================] - 86s 109ms/step - loss: 0.1537 - acc: 0.9479 - val_loss: 0.4459 - val_acc: 0.8669
Epoch 40/70
782/782 [==============================] - 86s 110ms/step - loss: 0.1435 - acc: 0.9502 - val_loss: 0.4997 - val_acc: 0.8556
Epoch 41/70
782/782 [==============================] - 86s 110ms/step - loss: 0.1458 - acc: 0.9499 - val_loss: 0.5029 - val_acc: 0.8600
Epoch 42/70
782/782 [==============================] - 86s 110ms/step - loss: 0.1385 - acc: 0.9521 - val_loss: 0.5368 - val_acc: 0.8647
Epoch 43/70
782/782 [===========================